<a href="https://colab.research.google.com/github/HSS107048212/ELEBC_Youbike/blob/main/05_AC_PV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get Excel in the folder

In [29]:
# 讀取資料夾中的excel檔名
import os
import pandas as pd
# 指定你的目錄路徑
directory_path = '/content/'
# C:/Users/Elebc_GPU_Server/Desktop/排程抓資料/
# /content/


# 獲取目錄中的所有檔案和子目錄名稱
files_and_directories = os.listdir(directory_path)
print(files_and_directories)
file=[]
for item in files_and_directories:
    # 檢查是否包含指定日期和文件名
    if "0015_Youbike_cleaned" in item:
        print("Matched File:", item)
        file.append(item)

# 打印出列表
print(file)

['.config', '2024-04-14 0015_Youbike_cleaned.xlsx', '2024-04-15 merged_data.xlsx', '2024-04-13 0015_Youbike_cleaned.xlsx', '04-22 EV_data.xlsx', '2024-04-13 0530_Youbike_cleaned.xlsx', 'rich-operand-418201-63c868c738c8.json', '2024-04-14 merged_data.xlsx', '2024-04-13 merged_data.xlsx', 'sample_data']
Matched File: 2024-04-14 0015_Youbike_cleaned.xlsx
Matched File: 2024-04-13 0015_Youbike_cleaned.xlsx
['2024-04-14 0015_Youbike_cleaned.xlsx', '2024-04-13 0015_Youbike_cleaned.xlsx']


## Filter the start day (UI: days)

In [30]:
import os
import datetime

# 獲取今天的日期
today = datetime.date.today()

# 計算 ? 天前的日期開始的資料
## 後續會接for迴圈，所以假設days=5，那麼前5天，前4天，前3天，前2天，前1天都會被計算。
## 如果days=1，那麼就是算前1天。
days_ago = today - datetime.timedelta(days=11) #後續更改----------------------------------------------------------------------------

# 將日期格式化為 YYYY-MM-DD
formatted_date = days_ago.strftime('%Y-%m-%d')
print(formatted_date)


# 排序文件名列表，假設日期和時間格式固定，可以直接按字符串排序
matched_files = [f for f in file if formatted_date <= f ]
matched_files = sorted(matched_files)

# 打印出匹配的文件名
print(matched_files)

2024-04-13
['2024-04-13 0015_Youbike_cleaned.xlsx', '2024-04-14 0015_Youbike_cleaned.xlsx']


## Processing Function: Compute AC (UI: Rides_day_for_breakeven)

In [31]:
import numpy as np
def ac (df):
  Rides_day_for_breakeven = 93667
  # Why Rides_day_for_breakeven is 93667? you can check on https://docs.google.com/presentation/d/1tBNcwg75QwaClUM5m2wP-YxiqWZI_SR3Ztjk6YZLKsw/edit#slide=id.g26e84c87d13_0_209
  df["AC"] = np.ceil(Rides_day_for_breakeven * df["tot"] / sum(df["tot"]))
  return df

## Processing Function: Compute PV (UI: Expected_usage_percentage)

In [32]:
def pv (df):
  Expected_usage_percentage = 0.05
  # Why Expected_usage_percentage is 0.05 ? you can check on https://docs.google.com/presentation/d/1tBNcwg75QwaClUM5m2wP-YxiqWZI_SR3Ztjk6YZLKsw/edit#slide=id.g26e84c87d13_0_242
  df["PV"] = np.ceil(df["tot"] * Expected_usage_percentage *95)
  return df

## Import AC_PV file to Google drive

In [33]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# 設置Google Drive API的認證
credentials = service_account.Credentials.from_service_account_file(
    f'{directory_path}rich-operand-418201-63c868c738c8.json',  # 請替換成你的認證文件路徑
    scopes=['https://www.googleapis.com/auth/drive']
)

# 創建Google Drive API的服務
drive_service = build('drive', 'v3', credentials=credentials)
# 要上傳到的Google Drive文件夾的ID
folder_id = '1aeTv-6l0nfmpEAfKHBvd7l5p0l9Kfplb'  # 請替換成目標文件夾的ID



## Main Function: Input Youbike 2.0 Data to Processing Function

In [35]:
files = [(f"{directory_path}{filename}", filename[0:10]) for filename in matched_files]
print(files)
print()

for file_path, time_label in files:
    print(file_path)
    print(time_label)
    # Read data from file
    data = pd.read_excel(file_path)
    selected_columns = ['sno', 'sna',"sarea","ar","lat","lng",'tot','infoDate']
    # 创建新的DataFrame，只包含选定的列
    data= data[selected_columns]

    AC_data = ac(data)
    PV_data = pv(data)

    PV_data.to_excel(f"{time_label} AC_PV.xlsx", index=False)  # index=False to avoid writing row indices to the file

    # 要上傳的Excel文件路徑
    excel_file_path = f'{directory_path}{time_label} AC_PV.xlsx'  # 請替換成你的Excel文件路徑
    import os
    # 上傳文件
    file_metadata = {
        'name': os.path.basename(excel_file_path),
        'parents': [folder_id]
    }
    media = MediaFileUpload(excel_file_path, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print('File ID: %s' % file.get('id'))

[('/content/2024-04-13 0015_Youbike_cleaned.xlsx', '2024-04-13'), ('/content/2024-04-14 0015_Youbike_cleaned.xlsx', '2024-04-14')]

/content/2024-04-13 0015_Youbike_cleaned.xlsx
2024-04-13
File ID: 1QGu5GAbO4zaaCEVL3kBh-Pru0khLACdt
/content/2024-04-14 0015_Youbike_cleaned.xlsx
2024-04-14
File ID: 1FfFwNSy2hu40Fx4uPUhHGmKGrVnvc1PE
